
# Predicción de nivel de estrés a partir de horas de sueño (Regresión lineal)

Este notebook **didáctico** muestra, paso a paso y con explicaciones sencillas, cómo:
1. **Cargar un conjunto de datos** (horas de sueño y nivel de estrés).
2. **Explorar** rápidamente la información.
3. **Entrenar** un modelo de **regresión lineal**.
4. **Evaluar** su desempeño con métricas simples.
5. **Visualizar** la recta de regresión sobre los datos.
6. **Usar el modelo** para hacer predicciones.
7. **Guardar** el modelo entrenado para reutilizarlo.

> **Sobre los datos**: Este notebook incluye un archivo pequeño `sueño_estress_dataset.csv` (≤100 filas) para demo.
> Para tu proyecto, **te recomiendo usar datos reales**. Un conjunto muy usado es el **Sleep Health and Lifestyle Dataset** (encuesta con columnas *Sleep Duration* y *Stress Level*).
> Puedes descargarlo y reemplazar el archivo de ejemplo por el real. Más abajo verás cómo hacerlo.


## Imports y librerias

In [ ]:

# =======================
# IMPORTAR LIBRERÍAS
# =======================
# pandas: para cargar y manipular datos tabulares (CSV, DataFrames)
import pandas as pandas_library
# numpy: utilidades numéricas (arreglos, operaciones matemáticas)
import numpy as numpy_library
# matplotlib.pyplot: graficación básica (dispersión, líneas, títulos, etc.)
import matplotlib.pyplot as pyplot_library
# scikit-learn: librería de ML con modelos, métricas y utilidades
from sklearn.model_selection import train_test_split  # separar datos en entrenamiento/prueba
from sklearn.linear_model import LinearRegression      # modelo de regresión lineal
# joblib: guardar/cargar modelos entrenados a disco
import joblib as joblib_library


## Preparar los datos

In [ ]:

# =======================
# CARGAR LOS DATOS
# =======================
# Este CSV de demo tiene dos columnas:
#  - 'sleep_hours': horas de sueño por noche
#  - 'stress_level_1_to_10': nivel de estrés reportado (1= bajo, 10= alto)
# Si ya descargaste un dataset REAL (por ejemplo el de Kaggle), cambia 'csv_path' a su ruta local.
csv_path = r"sueño_estress_dataset.csv"  # <-- Cambia esta ruta si usarás tus propios datos
dataset_table = pandas_library.read_csv(csv_path)

# Mostramos las primeras filas para confirmar estructura y contenido
dataset_table.head()


In [ ]:

# =======================
# INSPECCIÓN RÁPIDA
# =======================
# Información general: columnas, tipos de datos, filas no nulas
# dataset_table_info = dataset_table.info()
dataset_table.describe()



In [ ]:

# =======================
# LIMPIEZA BÁSICA (si fuera necesaria)
# =======================
# Para mantener el ejemplo simple:
# - Si existieran valores faltantes en cualquiera de las columnas clave, los eliminamos.
columns_required = ["sleep_hours", "stress_level_1_to_10"]
dataset_table_clean = dataset_table.dropna(subset=columns_required).copy()

# Confirmamos el tamaño después de limpiar
dataset_table_clean.shape


## Entrenando el Modelo

In [ ]:

# =======================
# ENTRENAR EL MODELO DE REGRESIÓN LINEAL
# =======================
feature_series = dataset_table_clean[["sleep_hours"]]         # Features
label_series = dataset_table_clean["stress_level_1_to_10"]   # Labels

linear_regression_model = LinearRegression()

# Aca es donde entrenamos el modelo realmente
linear_regression_model.fit(feature_series, label_series)

# Coeficientes aprendidos por el modelo
learned_intercept = float(linear_regression_model.intercept_)
learned_slope = float(linear_regression_model.coef_[0])

print(f"Intercept (b0): {learned_intercept}")
print(f"Slope (b1): {learned_slope}")


## Gráfica de dispersión y regresión lineal

In [ ]:

# =======================
# VISUALIZAR LA RECTA DE REGRESIÓN
# =======================
## Inicializa el Gráfico
pyplot_library.figure()
## Titulos y descripciones
pyplot_library.title("Relación horas de sueño vs. nivel de estrés (Regresión lineal)")
pyplot_library.xlabel("Horas de sueño por noche")
pyplot_library.ylabel("Nivel de estrés (1=bajo, 10=alto)")

## crea los puntos de dispersion de los datos de entrenamiento
pyplot_library.scatter(dataset_table_clean["sleep_hours"], dataset_table_clean["stress_level_1_to_10"], label="Datos observados")

## ordenamos los datos de las features y dibujamos la Recta de regresión
ordered_sleep = dataset_table_clean["sleep_hours"].sort_values()
predicted_line = linear_regression_model.predict(ordered_sleep.to_frame())
pyplot_library.plot(ordered_sleep, predicted_line, label="Recta de regresión")

## Renderiza el gráfico
pyplot_library.legend()
pyplot_library.show()


In [ ]:

# =======================
# INTERPRETACIÓN SIMPLE DEL MODELO
# =======================
print(f"Intercepto (b0): {learned_intercept:.3f}")
print(f"Pendiente (b1):  {learned_slope:.3f}")
if learned_slope < 0:
    print("Interpretación: A más horas de sueño, el modelo espera MENOR nivel de estrés (pendiente negativa).")
elif learned_slope > 0:
    print("Interpretación: A más horas de sueño, el modelo espera MAYOR nivel de estrés (pendiente positiva).")
else:
    print("Interpretación: El modelo no encontró relación lineal entre horas de sueño y estrés (pendiente ~ 0).")


## Predecir nuevos valores (features)

In [ ]:

# =======================
# HACER PREDICCIONES CON EL MODELO
# =======================
# Datos a predecir, puede ser uno o varios
new_feature_data = [10]

#convierte los datos a Dataframe
new_sleep_hours = pandas_library.DataFrame({"sleep_hours": new_feature_data})
#Hace la prediccion usando el modelo entrenado
predicted_stress_levels = linear_regression_model.predict(new_sleep_hours)

#para cada feature imprimir su valor de prediccion
for hours, pred in zip(new_sleep_hours["sleep_hours"], predicted_stress_levels):
    print(f"Horas de sueño: {hours:.1f} -> Estrés estimado: {pred:.2f} (1=bajo, 10=alto)")


## (Bonus!) Exportar mi modelo!

In [ ]:
# =======================
# INSTALAR LIBRERÍAS DE ONNX
# =======================
# Estas librerías permiten convertir modelos de scikit-learn (skl) a ONNX
!pip install skl2onnx onnxruntime

In [ ]:
# =======================
# EXPORTAR EL MODELO A ONNX
# =======================

from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import numpy as np

# 1. Definir la firma de la entrada (Initial Types)
# El modelo espera una entrada de 1 feature (sleep_hours).
# La forma [None, 1] significa: cualquier número de filas (None) y 1 columna (nuestra feature).
initial_type = [('sleep_hours', FloatTensorType([None, 1]))]

# 2. Convertir el modelo de scikit-learn a ONNX
# target_opset=13 es una versión estable y compatible
onnx_model = convert_sklearn(
    linear_regression_model,
    initial_types=initial_type,
    target_opset=13
)

# 3. Guardar el modelo ONNX a un archivo binario
onnx_file_path = "exported_model.onnx"
with open(onnx_file_path, "wb") as f:
    f.write(onnx_model.SerializeToString())

print(f"✅ ¡Modelo exportado con éxito a ONNX!")
print(f"Archivo guardado en: {onnx_file_path}")
print("\n¡Ya puedes usar este archivo .onnx en tu proyecto de Node.js, C#, o cualquier entorno que soporte ONNX Runtime!")


### Ejemplo de como usarlo en un proyecto JS Node

***Instalamos dependencias onnx para Node***
```js
// Install deps
npm install onnxruntime-node
```


***ejemplo de una funcion que implementa el modelo***
```js
// Implementation in JS
// Importa la librería ONNX Runtime para Node.js
const ort = require('onnxruntime-node');

// Ruta de nuestro modelo exportado desde Python
const modelPath = 'linear_regression_model.onnx';

/**
 * Función que realiza la predicción de estrés
 * @param {number} sleepHours - Las horas de sueño (nuestra Feature de entrada)
 * @returns {Promise<number>} - El nivel de estrés predicho (nuestra Label de salida)
 */
async function predictStress(sleepHours) {
    
    // 1. Cargar el modelo ONNX
    const session = await ort.InferenceSession.create(modelPath);

    // 2. Preparar los datos de entrada
    // ONNX espera que el input sea un tensor (un array estructurado).
    // Nuestra entrada es una sola Feature (horas de sueño)
    
    // Definimos el nombre de la Feature que dimos en Python: 'sleep_hours'
    const inputName = 'sleep_hours';
    
    // Creamos el tensor: debe ser Float32 y tener la forma [1, 1]
    // [1] porque es una sola predicción, [1] porque es una sola feature.
    const inputTensor = new ort.Tensor('float32', [sleepHours], [1, 1]);

    // Mapeamos el nombre de la entrada al tensor de datos
    const inputFeed = { [inputName]: inputTensor };

    // 3. Ejecutar la inferencia (la predicción)
    const results = await session.run(inputFeed);
    
    // 4. Extraer el resultado
    // El modelo de regresión lineal solo tiene una salida (una Label)
    const outputName = session.outputNames[0];
    const stressLevel = results[outputName].data[0];
    
    return stressLevel;
}

// ======================================
// Ejemplo de Uso: Predecir niveles de estrés
// ======================================
const horas_dormir_caso1 = 5.5;
const horas_dormir_caso2 = 8.2;

(async () => {
    try {
        console.log("--- Modelo de Regresión Lineal (Node.js) ---");
        
        // Predicción 1
        const pred1 = await predictStress(horas_dormir_caso1);
        console.log(`Si alguien duerme ${horas_dormir_caso1} horas, el Estrés estimado es: ${pred1.toFixed(2)}`);

        // Predicción 2
        const pred2 = await predictStress(horas_dormir_caso2);
        console.log(`Si alguien duerme ${horas_dormir_caso2} horas, el Estrés estimado es: ${pred2.toFixed(2)}`);

        console.log("\n¡Así de simple se usa el modelo de Python en Node.js!");

    } catch (e) {
        console.error("Hubo un error al ejecutar la predicción:", e);
    }
})();
```



## Info adicional y referencias

### ¿Cómo usar un **dataset real** en lugar del de demo?

1. Descarga un dataset con **horas de sueño** y **nivel de estrés**. Por ejemplo:
   - **Sleep Health and Lifestyle Dataset** (encuesta con columnas *Sleep Duration* y *Stress Level*). Consulta la licencia antes de un uso comercial.
2. Guarda el archivo CSV en tu equipo, por ejemplo: `data/Sleep_health_and_lifestyle_dataset.csv`.
3. Cambia la variable `csv_path` en la celda **CARGAR LOS DATOS** para apuntar a tu archivo real y vuelve a ejecutar el notebook.
4. Si tus columnas se llaman distinto, ajusta:
   - `columns_required`
   - `feature_series` (característica)
   - `label_series` (objetivo)



### Referencias breves

- Sleep Health and Lifestyle Dataset (encuesta con *Sleep Duration* y *Stress Level*) disponible en portales públicos de datos.  
- Revisión general: existe evidencia de **relación negativa** entre horas de sueño y estrés en poblaciones adultas, observada en estudios de diarios y encuestas.
